# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install ..

Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.
Processing /Users/bytedance/Developer/todd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for todd-ai: filename=todd_ai-0.4.0-py3-none-any.whl size=104120 sha256=4a79610704328db1d032fd3043602dc99a2cac6fb4be42d0574fb362f6fb1142
  Stored in directory: /private/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/pip-ephem-wheel-cache-x01z44ue/wheels/15/ef/5a/9fc12e257ce5cef16b333a2ed6c992ff9cbcc9167f7199e6ac
Successfully built todd-ai

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pathlib
import tempfile
from pprint import pprint
from typing import Any, NoReturn, TypedDict, cast

import todd
import torch
import torch.nn.functional as F
import torch.utils.data

Memo = dict[str, Any]

/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[2023-07-26 19:05:05,868 86878:140704676111936][patches.py:72 todd <module>] INFO: `ipdb` is installed. Using it for debugging.


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register()
class RunnerModel(todd.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

### Runners

In [7]:
class RunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        y: torch.Tensor = self._strategy.model(batch['x'])
        loss = F.l1_loss(y, batch['y'])
        memo['loss'] = loss
        if 'log' in memo:
            memo['log']['loss'] = f'{loss.item():.3f}'
        return memo

In [8]:
class TrainerMixin(RunnerMixin):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        memo = super()._run_iter(batch, memo)
        if 'log' in memo:
            model = cast(RunnerModel, self._strategy.module)
            memo['log']['weight'] = f'{model.weight.item():.3f}'
            memo['log']['batch'] = str(batch)
        return memo

In [9]:
@todd.RunnerRegistry.register()
class CustomValidator(RunnerMixin, todd.runners.Validator):
    pass

In [10]:
@todd.RunnerRegistry.register()
class CustomIterBasedTrainer(TrainerMixin, todd.runners.IterBasedTrainer):
    pass

In [11]:
@todd.RunnerRegistry.register()
class CustomEpochBasedTrainer(TrainerMixin, todd.runners.EpochBasedTrainer):
    pass

## Validators

In [12]:
validator_demo = todd.Config(
    type='CustomValidator',
    name='custom_validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
    callbacks=dict(type='LogCallback', interval=5),
)

In [13]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-07-26 19:05:06,543 86878:140704676111936][base.py:52 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:06,545 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-26 19:05:06,547 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-26 19:05:06,549 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-26 19:05:06,550 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp4yhsm6_1
└── custom_validator

2 directories, 0 files


## Trainers

In [14]:
trainer_demo = validator_demo.copy()
trainer_demo.pop('type')
trainer_demo.dataloader = todd.Config(
    batch_size=2,
    shuffle=True,
    dataset=dict(type='RunnerDataset', n=67),
)
trainer_demo.optimizer = todd.Config(type='SGD', lr=0.005)


### Iteration Based

In [15]:
iter_based_trainer_demo = trainer_demo.copy()
iter_based_trainer_demo.type = 'CustomIterBasedTrainer'
iter_based_trainer_demo.name = 'custom_iter_based_trainer'
iter_based_trainer_demo.iters = 53

In [16]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        iter_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:06,863 86878:140704676111936][base.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:06,867 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=87.000 weight=0.000 batch={'x': tensor([59, 28]), 'y': tensor([118,  56])}
[2023-07-26 19:05:06,870 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=114.000 weight=0.000 batch={'x': tensor([49, 65]), 'y': tensor([ 98, 130])}
[2023-07-26 19:05:06,872 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=70.000 weight=0.000 batch={'x': tensor([20, 50]), 'y': tensor([ 40, 100])}
[2023-07-26 19:05:06,873 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [20/5

### Epoch Based

In [17]:
epoch_based_trainer_demo = trainer_demo.copy()
epoch_based_trainer_demo.type = 'CustomEpochBasedTrainer'
epoch_based_trainer_demo.name = 'custom_epoch_based_trainer'
epoch_based_trainer_demo.epochs = 3

In [18]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        epoch_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:06,947 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:06,951 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-26 19:05:06,954 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=75.000 weight=0.000 batch={'x': tensor([18, 57]), 'y': tensor([ 36, 114])}
[2023-07-26 19:05:06,956 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=45.000 weight=0.000 batch={'x': tensor([37,  8]), 'y': tensor([74, 16])}
[2023-07-26 19:05:06,958 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=108.000 weight=0.000 batch={'x': tensor([49, 59]), 'y': tenso

## Callbacks

### Log

In [19]:
log_callback_demo = validator_demo.copy()
log_callback = log_callback_demo.callbacks
log_callback.with_file_handler = True
log_callback_demo.callbacks = [log_callback]

In [20]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        log_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-07-26 19:05:07,022 86878:140704676111936][base.py:52 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,025 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-26 19:05:07,026 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-26 19:05:07,028 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-26 19:05:07,030 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp1r6nblpz
└── custom_validator
    └── 2023-07-26T19-05-07_022077-08-00.log

2 directories, 1 file

[2023-07-26 19:05:07,022 86878:140704676111936][base.py:52 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,025 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] loss=10.000
[2023-07-26 19:05:07,026 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] loss=20.000
[2023-07-26 19:05:07,028 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] loss=30.000
[2023-07-26 19:05:07,030 86878:140704676111936][log.py:48 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] loss=40.000


### Optimize

In [21]:
optimize_callback_demo = iter_based_trainer_demo.copy()
optimize_callback = todd.Config(type='OptimizeCallback')
optimize_callback_demo.callbacks = [optimize_callback, log_callback]

In [22]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        optimize_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:07,590 86878:140704676111936][base.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,594 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=44.240 weight=0.618 batch={'x': tensor([14, 50]), 'y': tensor([ 28, 100])}
[2023-07-26 19:05:07,597 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=5.445 weight=1.395 batch={'x': tensor([15,  3]), 'y': tensor([30,  6])}
[2023-07-26 19:05:07,600 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=0.963 weight=1.975 batch={'x': tensor([40, 37]), 'y': tensor([80, 74])}
[2023-07-26 19:05:07,603 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [20/53] loss

### Learning Rate Schedule

In [23]:
lr_schedule_callback_demo = iter_based_trainer_demo.copy()
lr_schedule_callback = todd.Config(
    type='LRScheduleCallback',
    lr_scheduler=dict(type='LinearLR', total_iters=10),
)
lr_schedule_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_callback,
    log_callback,
]


In [24]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:07,637 86878:140704676111936][base.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,640 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=58.149 weight=0.362 batch={'x': tensor([56, 15]), 'y': tensor([112,  30])} lr=['3.333e-03']
[2023-07-26 19:05:07,643 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=64.715 weight=0.930 batch={'x': tensor([54, 67]), 'y': tensor([108, 134])} lr=['5.000e-03']
[2023-07-26 19:05:07,646 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=13.367 weight=1.733 batch={'x': tensor([59, 41]), 'y': tensor([118,  82])} lr=['5.000e-03']
[2023-07-26 19:05:07,650 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_i

In [25]:
lr_schedule_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
lr_schedule_by_epoch_callback = lr_schedule_callback.copy()
lr_schedule_by_epoch_callback.by_epoch = True
lr_schedule_by_epoch_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_by_epoch_callback,
    log_callback,
]


In [26]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_by_epoch_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:07,685 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,686 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-26 19:05:07,690 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=19.021 weight=0.271 batch={'x': tensor([ 2, 20]), 'y': tensor([ 4, 40])} lr=['1.667e-03']
[2023-07-26 19:05:07,693 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=29.605 weight=0.556 batch={'x': tensor([37,  4]), 'y': tensor([74,  8])} lr=['1.667e-03']
[2023-07-26 19:05:07,696 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=23.924 weight=0.741 batch={'x

### Learning Rate Scaler

In [27]:
lr_scaler_callback_demo = iter_based_trainer_demo.copy()
lr_scaler_callback = todd.Config(
    type='LRScaleCallback',
    lr_scaler=dict(base_batch_size=1),
)
lr_scaler_callback_demo.callbacks = [
    optimize_callback,
    lr_scaler_callback,
    log_callback,
]

In [28]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_scaler_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:07,780 86878:140704676111936][lr.py:92 todd.CustomIterBasedTrainer.custom_iter_based_trainer _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-07-26 19:05:07,782 86878:140704676111936][base.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,786 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=31.785 weight=1.185 batch={'x': tensor([43, 35]), 'y': tensor([86, 70])}
[2023-07-26 19:05:07,789 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=8.140 weight=2.185 batch={'x': tensor([61, 27]), 'y': tensor([122,  54])}
[2023-07-26 19:05:07,792 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=2.450 weight=2.140 batch={'x': tensor([33,  2]),

### Checkpoint

In [29]:
checkpoint_callback_demo = iter_based_trainer_demo.copy()
checkpoint_callback = todd.Config(type='CheckpointCallback', interval=10)
checkpoint_callback_demo.callbacks = [checkpoint_callback, log_callback]

In [30]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_50 = pathlib.Path(work_dirs) / 'custom_iter_based_trainer' / 'checkpoints' / 'iter_50'
    for f in iter_50.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-07-26 19:05:07,832 86878:140704676111936][base.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:07,834 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=90.000 weight=0.000 batch={'x': tensor([44, 46]), 'y': tensor([88, 92])}
[2023-07-26 19:05:07,837 86878:140704676111936][checkpoint.py:59 todd.CustomIterBasedTrainer.custom_iter_based_trainer _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp0f8g7ela/custom_iter_based_trainer/checkpoints/iter_10
[2023-07-26 19:05:07,839 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=75.000 weight=0.000 batch={'x': tensor([26, 49]), 'y': tensor([52, 98])}
[2023-07-26 19:05:07,842 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer afte


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp0f8g7ela
└── custom_iter_based_trainer
    ├── 2023-07-26T19-05-07_831637-08-00.log
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_20
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_30
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_40
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_50
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├─

In [31]:
checkpoint_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
checkpoint_by_epoch_callback = checkpoint_callback.copy()
checkpoint_by_epoch_callback.update(interval=1, by_epoch=True)
checkpoint_by_epoch_callback_demo.callbacks = [
    checkpoint_by_epoch_callback,
    log_callback,
]

In [32]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_by_epoch_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'custom_epoch_based_trainer' / 'checkpoints' / 'epoch_2'
    for f in epoch_2.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-07-26 19:05:08,355 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:08,356 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-26 19:05:08,360 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=67.000 weight=0.000 batch={'x': tensor([45, 22]), 'y': tensor([90, 44])}
[2023-07-26 19:05:08,363 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=45.000 weight=0.000 batch={'x': tensor([ 9, 36]), 'y': tensor([18, 72])}
[2023-07-26 19:05:08,367 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=83.000 weight=0.000 batch={'x': tensor([55, 28]), 'y': tensor([


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpxfqj6q1q
└── custom_epoch_based_trainer
    ├── 2023-07-26T19-05-08_355022-08-00.log
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├── meta.pth
            ├── model.pth
            ├── optim.pth
            └── strategy.pth

7 directories, 21 files

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                 

In [33]:
checkpoint_load_from_callback_demo = checkpoint_by_epoch_callback_demo.copy()
checkpoint_load_from_callback_demo.callbacks = [
    optimize_callback,
    checkpoint_by_epoch_callback,
    log_callback,
]

In [34]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
        load_from=os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2')
    )
    runner.run()

[2023-07-26 19:05:09,046 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:09,048 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-26 19:05:09,053 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=35.100 weight=0.700 batch={'x': tensor([ 1, 53]), 'y': tensor([  2, 106])}
[2023-07-26 19:05:09,057 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=19.035 weight=1.595 batch={'x': tensor([59, 35]), 'y': tensor([118,  70])}
[2023-07-26 19:05:09,061 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=0.569 weight=1.982 batch={'x': tensor([19, 46]), 'y': tenso


--------------------



[2023-07-26 19:05:09,565 86878:140704676111936][checkpoint.py:44 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer connect] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp4r7tm9jv/custom_epoch_based_trainer/checkpoints/epoch_2
[2023-07-26 19:05:09,571 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:09,572 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [3/3]
[2023-07-26 19:05:09,576 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [70/102] loss=3.847 weight=1.865 batch={'x': tensor([15, 42]), 'y': tensor([30, 84])}
[2023-07-26 19:05:09,579 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [75/102] loss=5.580 weight=2.155 batch={'x': ten

### Monitor

In [35]:
class CustomError(RuntimeError):
    pass

In [36]:
class FaultyRunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError('faulty runner')

In [37]:
@todd.RunnerRegistry.register()
class FaultyValidator(FaultyRunnerMixin, todd.runners.Validator):
    pass

In [38]:
@todd.RunnerRegistry.register()
class FaultyIterBasedTrainer(FaultyRunnerMixin, todd.runners.IterBasedTrainer):
    pass

In [39]:
@todd.RunnerRegistry.register()
class FaultyEpochBasedTrainer(
    FaultyRunnerMixin,
    todd.runners.EpochBasedTrainer,
):
    pass

In [40]:
monitor_callback_demo = validator_demo.copy()
monitor_callback_demo.type = 'FaultyValidator'
monitor_callback = todd.Config(type='MonitorCallback')
monitor_callback_demo.callbacks = [monitor_callback, log_callback]

In [41]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        monitor_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-07-26 19:05:09,677 86878:140704676111936][base.py:52 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:09,679 86878:140704676111936][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x154bc7390>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 192, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_86878/2137902126.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner



[2023-07-26 19:05:09,677 86878:140704676111936][base.py:52 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:09,679 86878:140704676111936][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x154bc7390>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 192, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_86878/2137902126.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner


### ETA

In [42]:
eta_callback_demo = iter_based_trainer_demo.copy()
eta_callback = todd.Config(type='ETACallback')
eta_callback_demo.callbacks = [eta_callback, log_callback]

In [43]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        eta_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-07-26 19:05:09,975 86878:140704676111936][base.py:52 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:09,979 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] loss=59.000 weight=0.000 batch={'x': tensor([ 7, 52]), 'y': tensor([ 14, 104])} eta=0:00:00.025862
[2023-07-26 19:05:09,983 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] loss=92.000 weight=0.000 batch={'x': tensor([30, 62]), 'y': tensor([ 60, 124])} eta=0:00:00.026458
[2023-07-26 19:05:09,985 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] loss=59.000 weight=0.000 batch={'x': tensor([ 8, 51]), 'y': tensor([ 16, 102])} eta=0:00:00.021797
[2023-07-26 19:05:09,988 86878:140704676111936][log.py:48 todd.CustomIterBasedTrainer.cu

### Priorities

## Strategies

In [44]:
strategy_load_model_from_demo = checkpoint_load_from_callback_demo.copy()

In [45]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.load_model_from(os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2', 'model.pth'))
    runner.run()

[2023-07-26 19:05:10,021 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:10,022 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-26 19:05:10,025 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=74.160 weight=0.560 batch={'x': tensor([54, 49]), 'y': tensor([108,  98])}
[2023-07-26 19:05:10,028 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=13.413 weight=1.537 batch={'x': tensor([36, 22]), 'y': tensor([72, 44])}
[2023-07-26 19:05:10,033 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] loss=5.606 weight=1.837 batch={'x': tensor([45, 24]), 'y': tensor(


--------------------



[2023-07-26 19:05:10,559 86878:140704676111936][base.py:52 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-07-26 19:05:10,571 86878:140704676111936][base.py:60 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpl4yrcj4_/custom_epoch_based_trainer/checkpoints/epoch_2/model.pth
[2023-07-26 19:05:10,577 86878:140704676111936][log.py:54 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-07-26 19:05:10,586 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] loss=1.508 weight=2.045 batch={'x': tensor([22, 45]), 'y': tensor([44, 90])}
[2023-07-26 19:05:10,599 86878:140704676111936][log.py:48 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] loss=4.046 weight=1.90

## Dry Run

In [46]:
todd.Store.DRY_RUN = True